# Amex - trojans - Version 2

## Schedule
- feature engineering 1.5 months
- model tuning 1 month

This is a starter code template for numerical variables. 

- Time features(the groupby logic) should be created based on the distribution of variables
- If the distribution is kind of wiggle, record it in the dictionary below

##### DDL: 7.7(Meet on 6.30)
##### Deliverables: A csv file/ pickle file + code

Upload Address: https://drive.google.com/drive/folders/1zyNIH8U4atqI8X48IFbeERpX65nVSRH5?usp=sharing

Features' Dictionary: https://docs.google.com/spreadsheets/d/1UsodGz_lkmzVmvweA_0YZZwRoB2VBpdAVuEuPtb38wI/edit?usp=sharing

- Hongyi: P,R,D

- Duanmu: 'D_' top 56

- Ziyi: 'B_'

- Haobo: NN Model

In [121]:
!pip install arch

In [122]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
import sklearn
from arch.unitroot import ADF


In [ ]:
# add data 
# Download: https://www.kaggle.com/datasets/munumbutt/amexfeather
# Or press "add data" and search "amexfeather", you will see AMEX-Feather-Dataset

train = pd.read_feather('../input/amexfeather/train_data.ftr')
test = pd.read_feather('../input/amexfeather/test_data.ftr')

In [ ]:
r_list = [_ for _ in train.columns if _.startswith('R_')]

In [ ]:
r_list  

In [ ]:
variable = 'R_8'

In [ ]:
tempTrain = train[['customer_ID','S_2',variable,'target']]
tempTrain.head()
del train
gc.collect()

In [ ]:
tempTest = test[['customer_ID','S_2',variable]]
tempTest.head()
del test
gc.collect()

In [ ]:
def dataTransform(df):
    df['S_2'] = pd.to_datetime(df['S_2'])
    df['Year'] = df['S_2'].apply(lambda x:x.year)
    df['Month'] = df['S_2'].apply(lambda x:x.month)
    df['Day'] = df['S_2'].apply(lambda x:x.day)
    df['Year_Month'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
#     df['Year_Month'] = df['Year_Month'].apply(lambda x: hash(x)%13)

dataTransform(tempTrain)
dataTransform(tempTest)

In [ ]:
gc.collect()

In [ ]:
tempTrain.info()

In [ ]:
tempTrain.head()

In [ ]:
tempTrain.isnull().sum()

In [ ]:
def plot_plot(df,col):
    sns.distplot(df[col])
    plt.show()
    sns.barplot(df[col])
    plt.show()
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    data = df[(df[col]>lower)&(df[col]<upper)]
    sns.distplot(data[data['target']==1][col],color='blue')
    sns.distplot(data[data['target']==0][col],color='orange')
    plt.show()

plot_plot(tempTrain,variable)

In [ ]:
tempTrain[tempTrain[variable]>(tempTrain[variable].quantile(0.99))]['target'].value_counts()

In [ ]:
tempTrain[tempTrain[variable]<=(tempTrain[variable].quantile(0.99))]['target'].value_counts()

In [ ]:
def plot_year_month(df,col):
    plt.figure(figsize=(12,6))
    df['Year_Month'] = pd.to_datetime(df['Year_Month']).dt.date
    data = df.groupby('Year_Month')[col].agg(['mean','median']).reset_index()
    sns.barplot(x='Year_Month', y='mean', data=data)
    plt.show()
    plt.figure(figsize=(12,6))
    sns.barplot(x='Year_Month', y='median', data=data)

plot_year_month(tempTrain,variable)

In [ ]:
plot_year_month(tempTrain[tempTrain['target']==0],variable)

In [ ]:
plot_year_month(tempTrain[tempTrain['target']==1],variable)

In [ ]:
import random
from collections import Counter
def plot_time_feature(df,col,default):
    randCust = df[df['target']==default]['customer_ID'].unique()[random.randint(1,1000)]
    data = df[df['customer_ID']==randCust]
    values = list(data[col].astype('float32'))
    print(values)
    print(Counter(data[col]))
    print('Target',data['target'].iloc[0])
    sns.lineplot(data = values)

plot_time_feature(tempTrain,variable,1)


In [ ]:
plot_time_feature(tempTrain,variable,1)

In [ ]:
plot_time_feature(tempTrain,variable,0)

In [ ]:
plot_time_feature(tempTrain,variable,0)

In [ ]:
# extract time feature/groupby logic
# provided apis below

# give more weights to the latest months
def time_linear_weight(df):
    _ = list(df)
    weights = [(idx+1)/len(_)*value for idx,value in enumerate(_)]
    return sum(weights)

# you can also define non-linear weight function here
# def time_nonlinearfunc(df):
#     _ = list(df)
#     weights = [...]
#     return ...

# how status changes
def time_trend(df):
    _ = list(df)
    n = len(_)
    half = max(1,int(n/2))
    return np.mean(_[-half:])/np.mean(_)

# ratio
def time_betaR(df):
    _ = list(df)
    try:
        return (_[-1]-_[0])/_[0]
    except:
        return np.nan

# difference
def time_betaD(df):
    _ = list(df)
    return _[-1]-_[0]

# range
def time_range(df):
    _ = list(df)
    return max(_)-min(_)

# mean/std
def time_cov(df):
    _ = list(df)
    return np.mean(_)/np.std(_)

# stationary
def stationary(df):
    _ = list(df.dropna())
    try:
        if len(_)>11:
            _ = list(df.dropna().diff())[1:]
            return float(ADF(_).summary().tables[0].data[1][1])<0.05
        else:
            return -1
    except:
        return -1


dfr5 = tempTrain.groupby('customer_ID')[variable].agg(['mean','median','last',time_linear_weight,time_trend,time_betaR,time_betaD,time_range,time_cov,stationary])
dfr5_ = tempTest.groupby('customer_ID')[variable].agg(['mean','median','last',time_linear_weight,time_trend,time_betaR,time_betaD,time_range,time_cov,stationary])

In [ ]:
dfr5['stationary'] = dfr5['stationary']+1-1     
dfr5_['stationary'] = dfr5_['stationary']+1-1                         

In [ ]:
# check features' distribution         

In [ ]:
targetgroup = tempTrain.groupby('customer_ID')['target'].mean()
targetgroup.columns = ['target']
targetgroup = pd.concat([dfr5,targetgroup],axis=1)

In [ ]:
plt.figure(figsize=(18,6))
sns.distplot(targetgroup[(targetgroup['target']==1) & (targetgroup['time_betaD'] < 0.01) & (targetgroup['time_betaD'] > -0.01)]['time_betaD'],color='blue')
sns.distplot(targetgroup[(targetgroup['target']==0) & (targetgroup['time_betaD'] < 0.01) & (targetgroup['time_betaD'] > -0.01)]['time_betaD'],color='orange')

In [ ]:
plt.figure(figsize=(18,6))
sns.distplot(targetgroup[(targetgroup['target']==1)& (targetgroup['mean'] < 0.1)]['mean'],color='blue')
sns.distplot(targetgroup[(targetgroup['target']==0)& (targetgroup['mean'] < 0.1)]['mean'],color='orange')

In [ ]:
plt.figure(figsize=(18,6))
sns.distplot(targetgroup[(targetgroup['target']==1)& (targetgroup['time_trend'] <10)]['time_trend'],color='blue')
sns.distplot(targetgroup[(targetgroup['target']==0)& (targetgroup['time_trend'] < 10)]['time_trend'],color='orange')

In [ ]:
sns.histplot(targetgroup[targetgroup['target']==1]['stationary'],color='blue')
sns.histplot(targetgroup[targetgroup['target']==0]['stationary'],color='orange')

In [ ]:
targetgroup[targetgroup['target']==1]['stationary'].value_counts()

In [ ]:
targetgroup[targetgroup['target']==0]['stationary'].value_counts()

In [ ]:
# now next variable
# plot_plot(tempTrain,'B_?')
# plot_year_month(tempTrain,'B_?')
# plot_year_month(tempTrain[tempTrain['target']==0],'B_?')
# plot_year_month(tempTrain[tempTrain['target']==1],'B_?')
# plot_time_feature(tempTrain,'B_?',1)
# plot_time_feature(tempTrain,'B_?',1)  
# plot_time_feature(tempTrain,'B_?',0)
# plot_time_feature(tempTrain,'B_?',0)
# dfb10 = tempTrain.groupby('customer_ID')['B_?'].agg(['mean','median','last',?])            
# dfb10_ = tempTest.groupby('customer_ID')['B_?'].agg(['mean','median','last',?])

In [ ]:
result = pd.concat([dfr5],axis=1)
result_ = pd.concat([dfr5_],axis=1)

In [ ]:
def reduce_size(df):
    for i in df.columns:
        df[i] = df[i].astype('float16')

reduce_size(result)
reduce_size(result_)

In [ ]:
result.info()

In [ ]:
result.to_pickle('shy_train_r8.pkl')
result_.to_pickle('shy_test_r8_.pkl')